In [70]:
from datetime import datetime, timedelta
from collections import defaultdict

def get_month_range(target_month):
    start_date = datetime.strptime(target_month, "%Y-%m")
    year = start_date.year
    month = start_date.month
    if month == 12:
        end_date = datetime(year + 1, 1, 1) - timedelta(days=1)
    else:
        end_date = datetime(year, month + 1, 1) - timedelta(days=1)
    return start_date, end_date

def get_overlap(start1, end1, start2, end2):
    latest_start = max(start1, start2)
    earliest_end = min(end1, end2)
    if latest_start <= earliest_end:
        return latest_start, earliest_end
    return None, None

def generate_monthly_bill(item_list, target_month):
    month_start, month_end = get_month_range(target_month)
    grouped_items = defaultdict(lambda: {'qty': 0, 'amount': 0})
    line_items = []

    for item in item_list:
        item_start = datetime.strptime(item['start_date'], "%Y-%m-%d")
        item_end = datetime.strptime(item['stop_date'], "%Y-%m-%d")
        overlap_start, overlap_end = get_overlap(item_start, item_end, month_start, month_end)
        if not overlap_start:
            continue

        active_days = (overlap_end - overlap_start).days + 1
        month_days = (month_end - month_start).days + 1
        prorate_factor = active_days / month_days

        qty = int(item['qty'])
        rate = float(item['rate'])
        amount = qty * rate * prorate_factor

        billing_period = f"{overlap_start.date()} to {overlap_end.date()}"
        key = (item['item_code'], rate, billing_period)
        grouped_items[key]['qty'] += qty
        grouped_items[key]['amount'] += round(amount, 2)

    total_revenue = 0.0
    for (item_code, rate, billing_period), values in grouped_items.items():
        line_item = {
            "item_code": item_code,
            "rate": rate,
            "qty": values['qty'],
            "amount": round(values['amount'], 2),
            "billing_period": billing_period
        }
        line_items.append(line_item)
        total_revenue += values['amount']

    return {
        "line_items": line_items,
        "total_revenue": round(total_revenue, 2)
    }


In [71]:
item_list = [
    {"idx": 1, "item_code": "Executive Desk (4*2)", "sales_description": "Dedicated Executive Desk", "qty": 10, "rate": "1000", "amount": "10000", "start_date": "2023-11-01", "stop_date": "2024-10-17"},
    {"idx": 2, "item_code": "Executive Desk (4*2)", "qty": "10", "rate": "1080", "amount": "10800", "start_date": "2024-10-18", "stop_date": "2025-10-31"},
    {"idx": 3, "item_code": "Executive Desk (4*2)", "qty": 15, "rate": "1080", "amount": "16200", "start_date": "2024-11-01", "stop_date": "2025-10-31"},
    {"idx": 4, "item_code": "Executive Desk (4*2)", "qty": 5, "rate": "1000", "amount": "5000", "start_date": "2024-11-01", "stop_date": "2025-10-31"},
    {"idx": 5, "item_code": "Manager Cabin", "qty": 5, "rate": 5000, "amount": 25000, "start_date": "2024-11-01", "stop_date": "2025-10-31"},
    {"idx": 6, "item_code": "Manager Cabin", "qty": 7, "rate": "5000", "amount": 35000, "start_date": "2024-12-15", "stop_date": "2025-10-31"},
    {"idx": 7, "item_code": "Manager Cabin", "qty": 10, "rate": 4600, "amount": 46000, "start_date": "2023-11-01", "stop_date": "2024-10-17"},
    {"idx": 8, "item_code": "Parking (2S)", "qty": 10, "rate": 1000, "amount": 10000, "start_date": "2024-11-01", "stop_date": "2025-10-31"},
    {"idx": 9, "item_code": "Parking (2S)", "qty": 10, "rate": 0, "amount": 0, "start_date": "2024-11-01", "stop_date": "2025-10-31"},
    {"idx": 10, "item_code": "Executive Desk (4*2)", "qty": "8", "rate": "1100", "amount": "8800", "start_date": "2024-11-15", "stop_date": "2025-01-31"},
    {"idx": 11, "item_code": "Manager Cabin", "qty": "3", "rate": "5200", "amount": "15600", "start_date": "2024-10-10", "stop_date": "2024-11-10"},
    {"idx": 12, "item_code": "Conference Table", "qty": 1, "rate": "20000", "amount": "20000", "start_date": "2024-11-05", "stop_date": "2024-11-20"},
    {"idx": 13, "item_code": "Parking (2S)", "qty": 5, "rate": "1000", "amount": "5000", "start_date": "2024-11-15", "stop_date": "2025-02-28"},
    {"idx": 14, "item_code": "Reception Desk", "qty": 2, "rate": "7000", "amount": "14000", "start_date": "2024-11-01", "stop_date": "2025-03-31"},
    {"idx": 15, "item_code": "Reception Desk", "qty": 1, "rate": "7000", "amount": "7000", "start_date": "2024-11-10", "stop_date": "2024-11-25"},
    {"idx": 16, "item_code": "Breakout Area", "qty": 3, "rate": "3000", "amount": "9000", "start_date": "2024-01-01", "stop_date": "2024-01-31"}
]


In [72]:
result = generate_monthly_bill(item_list, "2024-11")


In [73]:
import pprint
pprint.pprint(result)


{'line_items': [{'amount': 27000.0,
                 'billing_period': '2024-11-01 to 2024-11-30',
                 'item_code': 'Executive Desk (4*2)',
                 'qty': 25,
                 'rate': 1080.0},
                {'amount': 5000.0,
                 'billing_period': '2024-11-01 to 2024-11-30',
                 'item_code': 'Executive Desk (4*2)',
                 'qty': 5,
                 'rate': 1000.0},
                {'amount': 25000.0,
                 'billing_period': '2024-11-01 to 2024-11-30',
                 'item_code': 'Manager Cabin',
                 'qty': 5,
                 'rate': 5000.0},
                {'amount': 10000.0,
                 'billing_period': '2024-11-01 to 2024-11-30',
                 'item_code': 'Parking (2S)',
                 'qty': 10,
                 'rate': 1000.0},
                {'amount': 0.0,
                 'billing_period': '2024-11-01 to 2024-11-30',
                 'item_code': 'Parking (2S)',
                 